# Description of what this code does needs to go here
*Question: Traveltimes or travelspeeds?*

In [3]:
import config     # Module that contains all the global variables, filenames etc.
import pprint     # Module for pretty-printing 


'''Select path name for files depending on whether you are working on your PC or cluster'''
# path_for_data is an optional variable that defaults to 'path'. This is included to facilate a seperate path
# address for large data files

path = './Data_Files/'         # Use this if working on PC
path_for_data = '../../'       # Use this if working on PC and if data files are in a different folder

#path = './scratch/'           # Use this if working on cluster
#path_for_data = './scratch/'  # Use this if working on cluster and if data files are in a different folder

filenames = config.generate_filenames(path, path_for_data)

print('The following filenames have been loaded:')
pprint.pprint(filenames)

print('\n\nThe following constants have been loaded: (with other extra module built-ins)')
dir(globals()['config'])

The following filenames have been loaded:
{'HT_speeds': './Data_Files/HT_speeds.txt',
 'HT_speeds_axed': './Data_Files/HT_speeds_axed.txt',
 'HT_speeds_seed': './Data_Files/Seeded0,1/HT_speeds.txt',
 'HT_trips': './Data_Files/HT_trips.txt',
 'HT_trips_axed': './Data_Files/HT_trips_axed.txt',
 'HT_trips_axed_seed': './Data_Files/Seeded0,1/HT_trips_axed.txt',
 'HT_trips_seed': './Data_Files/Seeded0,1/HT_trips.txt',
 'W_speeds': './Data_Files/W_speeds.txt',
 'W_speeds_axed_seed': './Data_Files/Seeded0,1/HT_speeds_axed.txt',
 'W_speeds_seed': './Data_Files/Seeded0,1/W_speeds.txt',
 'W_trips': './Data_Files/W_trips.txt',
 'W_trips_seed': './Data_Files/Seeded0,1/W_trips.txt',
 'data_coo_form': '../../data_coo_form.txt',
 'data_traveltimes': '../../data_travel_times.csv',
 'data_trips': '../../data_trips.csv',
 'data_trips_transpose': '../../data_trips_transpose.csv',
 'empty_link_ids': './Data_Files/empty_link_ids.txt',
 'full_link_ids': './Data_Files/full_link_ids.txt',
 'full_link_speeds':

['EMPTY_LINKS',
 'FULL_LINKS',
 'HOURS_IN_YEAR',
 'MID_LINKS',
 'RANK',
 'SEEDED',
 'TOTAL_LINKS',
 'TRIPS',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'generate_filenames']

In [6]:
config.TRIPS

1

In [22]:
a=10
import numpy as np
from read_data import read_full_link_json
import Phase1      # This name needs to be changed

full_link_ids, D = read_full_link_json()  # Reads trips or travelspeeds depending on config.TRIPS value 

print('Full_link data has been read')
#print np.nanmin(D.flatten())
if config.SEEDED == 1:
    seed_W = 0
    seed_H = 1
elif config.SEEDED == 0:
    seed_W = None
    seed_H = None
else:
    print('Seed value invalid. Needs to be 0 or 1. Check config.py!')
    quit()
    
W, H, error = Phase1.find_signatures(D, rank = config.RANK, seed_W = seed_W, seed_H = seed_H)

NameError: name 'filenames' is not defined

In [ ]:
if config.TRIPS == 1:
    np.savetxt(filenames['W_trips'], W);                print('W_trips written')
    np.savetxt(filenames['HT_trips'], H.T);             print('HT_trips written')
    np.savetxt(filenames['HT_trips_axed'], axe_H(H.T)); print('HT_trips_axed written')

elif config.TRIPS == 0:
    np.savetxt(filenames['W_speeds'], W);                print('W_speeds written')
    np.savetxt(filenames['HT_speeds'], H.T);             print('HT_speeds written')
    np.savetxt(filenames['HT_speeds_axed'], axe_H(H.T)); print('HT_speeds_axed written')